In [1]:
import pandas as pd
import numpy as np

- **Pandas Merge Function**

In [2]:
customers = pd.DataFrame({
    'customer_id': [101, 102, 103, 104],
    'name': ['Alice', 'Bob', 'Charlie', 'Diana'],
    'segment': ['Gold', 'Silver', 'Gold', 'Bronze']
})

transactions = pd.DataFrame({
    'txn_id': [1, 2, 3, 4, 5, 6],
    'customer_id': [101, 101, 102, 105, 103, 103],
    'amount': [120.0, 80.0, 50.0, 200.0, 35.0, 65.0],
    'date': pd.to_datetime(['2025-12-01','2025-12-05','2025-12-02','2025-12-03','2025-12-04','2025-12-10'])
})


In [3]:
customers

,customer_id,name,segment
0,101,Alice,Gold
1,102,Bob,Silver
2,103,Charlie,Gold
3,104,Diana,Bronze


In [4]:
transactions

,txn_id,customer_id,amount,date
0,1,101,120.0,2025-12-01
1,2,101,80.0,2025-12-05
2,3,102,50.0,2025-12-02
3,4,105,200.0,2025-12-03
4,5,103,35.0,2025-12-04
5,6,103,65.0,2025-12-10


- **Exercise task**
    - Task 1: Join transactions with customer details so each transaction shows the customer’s name and segment.
    - Task 2: Show all customers, even with no transactions (left join customers→transactions).
    - Task 3: Find customers that have transactions but aren’t in the customer list.
    - Task 4: Total spend per customer, then attach customer info.
    - Task 5: Total spend per segment.



In [5]:
task1 = pd.merge(transactions,customers,on="customer_id",how="inner")
task1

,txn_id,customer_id,amount,date,name,segment
0,1,101,120.0,2025-12-01,Alice,Gold
1,2,101,80.0,2025-12-05,Alice,Gold
2,3,102,50.0,2025-12-02,Bob,Silver
3,5,103,35.0,2025-12-04,Charlie,Gold
4,6,103,65.0,2025-12-10,Charlie,Gold


In [6]:
task2 = pd.merge(customers,transactions,on="customer_id",how="left")
task2

,customer_id,name,segment,txn_id,amount,date
0,101,Alice,Gold,1.0,120.0,2025-12-01
1,101,Alice,Gold,2.0,80.0,2025-12-05
2,102,Bob,Silver,3.0,50.0,2025-12-02
3,103,Charlie,Gold,5.0,35.0,2025-12-04
4,103,Charlie,Gold,6.0,65.0,2025-12-10
5,104,Diana,Bronze,NaN,NaN,NaT


In [7]:
task3 = pd.merge(customers[["customer_id"]],transactions,on="customer_id",how="right",indicator=True)
task3

,customer_id,txn_id,amount,date,_merge
0,101,1,120.0,2025-12-01,both
1,101,2,80.0,2025-12-05,both
2,102,3,50.0,2025-12-02,both
3,105,4,200.0,2025-12-03,right_only
4,103,5,35.0,2025-12-04,both
5,103,6,65.0,2025-12-10,both


In [8]:
task3[task3["_merge"] == "right_only"][["customer_id","txn_id","amount","date"]]

,customer_id,txn_id,amount,date
3,105,4,200.0,2025-12-03


In [9]:
# task 4
grouped_tr = transactions.groupby("customer_id")["amount"].sum().reset_index()
grouped_tr

,customer_id,amount
0,101,200.0
1,102,50.0
2,103,100.0
3,105,200.0


In [10]:
task4 = pd.merge(customers,grouped_tr,on="customer_id",how="left")
task4.sort_values(by="amount",na_position="first")

,customer_id,name,segment,amount
3,104,Diana,Bronze,NaN
1,102,Bob,Silver,50.0
2,103,Charlie,Gold,100.0
0,101,Alice,Gold,200.0


In [11]:
task5 = pd.merge(customers,transactions,on="customer_id",how="inner")
task5

,customer_id,name,segment,txn_id,amount,date
0,101,Alice,Gold,1,120.0,2025-12-01
1,101,Alice,Gold,2,80.0,2025-12-05
2,102,Bob,Silver,3,50.0,2025-12-02
3,103,Charlie,Gold,5,35.0,2025-12-04
4,103,Charlie,Gold,6,65.0,2025-12-10


In [12]:
task5.groupby("segment")["amount"].sum().reset_index()

,segment,amount
0,Gold,300.0
1,Silver,50.0


## Data Integration
- Actors and Directors Who Cooperated At Least Three Times


In [13]:
actors_and_directors = pd.DataFrame({
    'actor_id': [1, 1, 1, 1, 1, 2, 2],
    'director_id': [1, 1, 1, 2, 2, 1, 1],
    'timestamp': [0, 1, 2, 3, 4, 5, 6]
})
actors_and_directors

,actor_id,director_id,timestamp
0,1,1,0
1,1,1,1
2,1,1,2
3,1,2,3
4,1,2,4
5,2,1,5
6,2,1,6


In [14]:
actors_and_directors1 = actors_and_directors.groupby(["actor_id","director_id"])["timestamp"].count().reset_index()

In [15]:
actors_and_directors1

,actor_id,director_id,timestamp
0,1,1,3
1,1,2,2
2,2,1,2


In [16]:
actors_and_directors1.loc[actors_and_directors1.timestamp >= 3,["actor_id","director_id"]]

,actor_id,director_id
0,1,1


- **Replace Employee ID With The Unique Identifier**

In [17]:
employees_data = [
    [1, 'Alice'],
    [7, 'Bob'],
    [11, 'Meir'],
    [90, 'Winston'],
    [3, 'Jonathan']
]
employees = pd.DataFrame(employees_data, columns=['id', 'name'])

employee_uni_data = [
    [3, 1],
    [11, 2],
    [90, 3]
]
employee_uni = pd.DataFrame(employee_uni_data, columns=['id', 'unique_id'])

In [18]:
employees

,id,name
0,1,Alice
1,7,Bob
2,11,Meir
3,90,Winston
4,3,Jonathan


In [19]:
employee_uni

,id,unique_id
0,3,1
1,11,2
2,90,3


In [20]:
res = pd.merge(employees,employee_uni,on="id",how="left")
res[["unique_id","name"]]

,unique_id,name
0,NaN,Alice
1,NaN,Bob
2,2.0,Meir
3,3.0,Winston
4,1.0,Jonathan


- **Students and Examinations**

In [21]:
students = pd.DataFrame({
    'student_id': [1, 2, 13, 6],
    'student_name': ['Alice', 'Bob', 'John', 'Alex']
})

# Create Subjects table
subjects = pd.DataFrame({
    'subject_name': ['Math', 'Physics', 'Programming']
})

# Create Examinations table
examinations = pd.DataFrame({
    'student_id': [1, 1, 1, 2, 1, 1, 13, 13, 13, 2, 1],
    'subject_name': ['Math', 'Physics', 'Programming', 'Programming', 'Physics', 
                    'Math', 'Math', 'Programming', 'Physics', 'Math', 'Math']
})

In [22]:
students

,student_id,student_name
0,1,Alice
1,2,Bob
2,13,John
3,6,Alex


In [23]:
examinations

,student_id,subject_name
0,1,Math
1,1,Physics
2,1,Programming
3,2,Programming
4,1,Physics
5,1,Math
6,13,Math
7,13,Programming
8,13,Physics
9,2,Math


In [24]:
std_sub = pd.merge(students,subjects,how='cross').sort_values(by=["student_id","subject_name"])
std_sub

,student_id,student_name,subject_name
0,1,Alice,Math
1,1,Alice,Physics
2,1,Alice,Programming
3,2,Bob,Math
4,2,Bob,Physics
5,2,Bob,Programming
9,6,Alex,Math
10,6,Alex,Physics
11,6,Alex,Programming
6,13,John,Math


In [25]:
exam_count = examinations.groupby(["student_id","subject_name"]).size().reset_index(name="attended_exams")
exam_count

,student_id,subject_name,attended_exams
0,1,Math,3
1,1,Physics,2
2,1,Programming,1
3,2,Math,1
4,2,Programming,1
5,13,Math,1
6,13,Physics,1
7,13,Programming,1


In [26]:
res = pd.merge(std_sub,exam_count,on=["student_id","subject_name"],how="left")
res

,student_id,student_name,subject_name,attended_exams
0,1,Alice,Math,3.0
1,1,Alice,Physics,2.0
2,1,Alice,Programming,1.0
3,2,Bob,Math,1.0
4,2,Bob,Physics,NaN
5,2,Bob,Programming,1.0
6,6,Alex,Math,NaN
7,6,Alex,Physics,NaN
8,6,Alex,Programming,NaN
9,13,John,Math,1.0


In [27]:
res["attended_exams"]=res.attended_exams.fillna(0)

In [28]:
res

,student_id,student_name,subject_name,attended_exams
0,1,Alice,Math,3.0
1,1,Alice,Physics,2.0
2,1,Alice,Programming,1.0
3,2,Bob,Math,1.0
4,2,Bob,Physics,0.0
5,2,Bob,Programming,1.0
6,6,Alex,Math,0.0
7,6,Alex,Physics,0.0
8,6,Alex,Programming,0.0
9,13,John,Math,1.0


- **Managers with at least five reports**

In [29]:
employee = pd.DataFrame({
    'id': [101, 102, 103, 104, 105, 106],
    'name': ['John', 'Dan', 'James', 'Amy', 'Anne', 'Ron'],
    'department': ['A', 'A', 'A', 'A', 'A', 'B'],
    'managerId': [np.nan, 101, 101, 101, 101, 101]
})
employee

,id,name,department,managerId
0,101,John,A,NaN
1,102,Dan,A,101.0
2,103,James,A,101.0
3,104,Amy,A,101.0
4,105,Anne,A,101.0
5,106,Ron,B,101.0


In [30]:
managers = employee.groupby("managerId").size().reset_index(name="count")

In [31]:
managers

,managerId,count
0,101.0,5


In [32]:
res_em = pd.merge(employee,managers,left_on="id",right_on="managerId",how="inner")
res_em

,id,name,department,managerId_x,managerId_y,count
0,101,John,A,NaN,101.0,5


In [33]:
res_col = res_em.loc[res_em["count"] >= 5]
# pd.DataFrame({"name":res_col})

In [34]:
res_col

,id,name,department,managerId_x,managerId_y,count
0,101,John,A,NaN,101.0,5


In [35]:
pd.DataFrame({"name":res_col.name})

,name
0,John


- **Sales person**

In [82]:
sales_person_data = {
    'sales_id': [1, 2, 3, 4, 5],
    'name': ['John', 'Amy', 'Mark', 'Pam', 'Alex'],
    'salary': [100000, 12000, 65000, 25000, 5000],
    'commission_rate': [6, 5, 12, 25, 10],
    'hire_date': ['4/1/2006', '5/1/2010', '12/25/2008', '1/1/2005', '2/3/2007']
}

company_data = {
    'com_id': [1, 2, 3, 4],
    'name': ['RED', 'ORANGE', 'YELLOW', 'GREEN'],
    'city': ['Boston', 'New York', 'Boston', 'Austin']
}

orders_data = {
    'order_id': [1, 2, 3, 4],
    'order_date': ['1/1/2014', '2/1/2014', '3/1/2014', '4/1/2014'],
    'com_id': [3, 4, 1, 1],
    'sales_id': [4, 5, 1, 4],
    'amount': [10000, 5000, 50000, 25000]
}

In [83]:
sales_person = pd.DataFrame(sales_person_data)
company = pd.DataFrame(company_data)
orders = pd.DataFrame(orders_data)

In [84]:
company

,com_id,name,city
0,1,RED,Boston
1,2,ORANGE,New York
2,3,YELLOW,Boston
3,4,GREEN,Austin


In [85]:
orders

,order_id,order_date,com_id,sales_id,amount
0,1,1/1/2014,3,4,10000
1,2,2/1/2014,4,5,5000
2,3,3/1/2014,1,1,50000
3,4,4/1/2014,1,4,25000


In [90]:
merge_com_or = pd.merge(company,orders,on="com_id",how="inner")
merge_com_or

,com_id,name,city,order_id,order_date,sales_id,amount
0,1,RED,Boston,3,3/1/2014,1,50000
1,1,RED,Boston,4,4/1/2014,4,25000
2,3,YELLOW,Boston,1,1/1/2014,4,10000
3,4,GREEN,Austin,2,2/1/2014,5,5000


In [92]:
red_orders = merge_com_or[merge_com_or.name == "RED"]
red_orders


,com_id,name,city,order_id,order_date,sales_id,amount
0,1,RED,Boston,3,3/1/2014,1,50000
1,1,RED,Boston,4,4/1/2014,4,25000


In [95]:
red_sales_ids = red_orders.sales_id
red_sales_ids

0    1
1    4
Name: sales_id, dtype: int64

In [98]:
res = sales_person[~sales_person.sales_id.isin(red_sales_ids)]
res

,sales_id,name,salary,commission_rate,hire_date
1,2,Amy,12000,5,5/1/2010
2,3,Mark,65000,12,12/25/2008
4,5,Alex,5000,10,2/3/2007


In [100]:
res[["name"]]

,name
1,Amy
2,Mark
4,Alex
